# 📈 판매 예측

---

## 배경
새로운 SageMaker AI Unified Studio의 주요 장점 중 하나는 여러 소스의 데이터를 통합할 수 있다는 것입니다. 이 노트북에서는 Lakehouse의 데이터를 가져와 판매 예측 모델을 학습하는 예제를 살펴보겠습니다.

다음 단계들을 진행하겠습니다:

- 🔍 과거 매출 데이터 조회
- 🔄 Amazon SageMaker 알고리즘에 입력할 데이터 변환
- 🤖 매출 예측을 위한 모델 학습
- 📈 상위 5개 제품 탐지하기 위한 내년 매출 예측


## 🛠️ 준비

먼저 노트북 전반에 걸쳐 사용할 Python 라이브러리들을 불러오고, 기본 카탈로그를 통해 프로젝트 데이터베이스의 이름을 가져오겠습니다.


In [ ]:
# 필요한 라이브러리 가져오기
from sagemaker_studio import Project
import lab_utilities

# 데이터베이스 이름 가져오기 및 프로젝트 데이터베이스 찾기
project = Project()
databases = project.connection().catalog().databases
project_db = next(db.name for db in databases if db.name.startswith('project_ml_'))

<div class="alert alert-block alert-info">
<center><b>‼️️ 중요:</b> 노트북의 다른 SQL 셀에서 사용하기 위해 다음 셀의 출력에서 데이터베이스 이름 <code>project_ml_...</code>를 기록해 두세요 ‼️️</center>
</div>

In [ ]:
print(project_db)

## 🔍 데이터 탐색

여기서는 SQL을 사용하여 과거 매출 데이터를 조회하고 검토하겠습니다.

데이터를 조회해 보겠습니다 (위에서 확인한 `Project DB` 값으로 `[REPLACE-VALUE-HERE]`를 변경하세요):


In [ ]:
%%sql project.athena
select * from "awsdatacatalog"."[REPLACE-VALUE-HERE]"."sales_table_enriched_w_campaigns"

판다스 DataFrame으로 쿼리 결과 변환하기

In [ ]:
data = _.to_pandas()

***

## 🔄 데이터 준비

데이터 준비 과정은 원시 매출 데이터를 정확한 매출 예측이 가능한 형식으로 변환합니다. 이 중요한 단계는 다음과 같습니다:

1. 📅 **시간별 매출 구성**: 매출을 시간 순으로 정렬하고 연도, 월, 요일 등의 시간 패턴을 추출하여 계절별 트렌드와 주간 구매 행동을 파악합니다.

2. 📈 **매출 이력 생성**: 각 제품과 채널의 최근 실적 지표를 계산하여 시스템이 매출 동향과 추세를 이해하도록 돕습니다.

3. 🔢 **카테고리의 수치화**: 비즈니스 카테고리(지역, 제품 유형, 판매 채널 등)의 고유한 특성을 유지한 채로 숫자 형식으로 변환합니다.

4. ⏳ **학습 타임라인 설정**: 데이터를 과거 학습 데이터, 최근 검증 데이터, 가장 최신의 테스트 데이터로 나누어 예측이 견고한 패턴을 기반으로 하면서도 최근 트렌드에 대해 검증되도록 합니다.

5. ⚖️ **수치 스케일 조정**: 가격과 비용 같은 값들을 비교 가능한 스케일로 조정하여 큰 숫자가 불균형한 중요도를 갖지 않도록 합니다.

**비즈니스 가치**: 이 과정을 통해 우리 시스템은 최대 1년 ahead의 매출을 높은 정확도로 예측하고, 계절별/지역별 패턴을 고려하며, 시장 변화에 적응하고, 재고 계획과 마케팅 타이밍에 대한 인사이트를 제공할 수 있습니다. 이는 제품 집중도, 시장 확장, 자원 투자에 대한 전략적 의사결정을 위해 원시 데이터를 실행 가능한 인텔리전스로 변환합니다.


예측 기간을 정의하는 것부터 시작하겠습니다. 예를 들어 1년으로 설정하겠습니다.

예측 기간 형식:
- 연: 예시 "1y"
- 월: 예시 "3m"
- 일: 예시 "15d"

In [ ]:
horizon = "1y"

이제 데이터셋을 준비하겠습니다

In [ ]:
# 데이터셋의 누락된 값(NaN)을 0으로 채우기
# 이후 처리 과정에서 문제를 일으킬 수 있는 null 값이 없도록 보장합니다
#data = data.fillna(0)

# 머신러닝 학습 및 평가를 위한 데이터 처리 및 분할
(
    X_train,
    X_val,
    X_test,
    y_train,
    y_val,
    y_test,
    feature_columns,
    scaler,
    df_encoded,
    future_dates,
) = lab_utilities.process_data(data, prediction_horizon=horizon)

---

# 🤖 모델 학습: 매출 예측 엔진 구축하기

모델 학습 과정은 준비된 데이터를 활용하여 미래 매출을 예측할 수 있는 강력한 예측 엔진을 구축합니다. 이 중요한 단계는 다음과 같습니다:

1. **학습 시스템 생성**: 매출 데이터의 복잡한 패턴을 찾고 이를 기반으로 정확한 예측을 수행하는 데 탁월한 특수 알고리즘(XGBoost)을 구축합니다.

2. **과거 데이터로 시스템 학습**: 수천 건의 과거 매출 거래를 분석하여 매출량에 가장 큰 영향을 미치는 요소들을 파악하며 시스템을 학습시킵니다.

3. **예측 정확도 테스트**: 실제 매출 수치를 이미 알고 있는 최근 데이터에 대해 예측을 수행하여, 미래 예측에 사용하기 전 시스템의 정확도를 측정합니다.

4. **주요 매출 동인 식별**: 계절성, 가격, 지역 차이 등 매출에 가장 큰 영향을 미치는 요소들을 파악하여 가치 있는 비즈니스 인사이트를 제공합니다.

**비즈니스 가치**: 이렇게 학습된 모델은 우리의 매출 예측 엔진이 되어, 경영진이 시장 수요를 예측하고, 재고를 최적화하며, 마케팅 캠페인을 계획하고, 자원을 더 효과적으로 할당할 수 있게 합니다. 또한 가장 영향력 있는 매출 요소들을 파악함으로써 비즈니스 성장을 위한 전략적 인사이트를 제공합니다.


In [ ]:
# 준비된 데이터를 사용하여 모델 학습하기
model = lab_utilities.model_train_eval(X_train, y_train, X_val, y_val, feature_columns)

---

# 📈 매출 예측: 미래 비즈니스 성과 전망

매출 예측 과정은 학습된 모델을 사용하여 제품, 지역, 기간별 미래 매출을 예측합니다. 이 중요한 단계는 다음과 같습니다:

1. **미래 시나리오 생성**: 가장 최근의 매출 데이터를 기반으로 템플릿을 구축하여 예측을 위한 현실적인 시작점을 설정합니다.

2. **시간 진행 시뮬레이션**: 미래 날짜를 반영하도록 모든 시간 관련 요소(요일, 월, 계절)를 업데이트하여 시스템이 계절별 패턴과 트렌드를 고려할 수 있게 합니다.

3. **시장 인텔리전스 적용**: 과거 패턴을 기반으로 이러한 변화하는 조건에 대한 매출 반응을 예측하기 위해 학습된 모델을 사용합니다.

4. **현실적 변동성 추가**: 주말 효과(매출 +15%)와 계절 요인(여름철 +10%)과 같은 자연스러운 변동을 포함하여 실제 시장 변동을 반영한 더 현실적인 예측을 만듭니다.

5. **의사결정을 위한 결과 정리**: 날짜, 지역, 제품 카테고리별로 예측을 정리하고 전략적 계획 수립을 위해 최고 실적 제품과 기간을 강조합니다.

**비즈니스 가치**: 이 예측 과정은 경영진이 미래 최대 판매 제품을 파악하고, 재고 수준을 최적화하며, 타겟 마케팅 캠페인을 계획하고, 지역과 제품 라인 전반에 걸쳐 자원을 효과적으로 할당할 수 있게 하는 실행 가능한 매출 예측을 제공합니다. 예측에 반영된 현실적인 변동성은 최상의 시나리오와 최악의 시나리오에 대비할 수 있도록 돕습니다.

유럽 지역의 내년 매출을 예측해 보겠습니다.


In [ ]:
# 분석 대상 지역 설정
region = "Europe"

# 해당 지역의 매출 예측 생성
product_sales = lab_utilities.predict_sales(
    model, df_encoded, future_dates, target_region=region
)

# 제품별 예측 총 매출 요약 생성
product_summary = (
    product_sales.groupby(["region", "item_type", "product_category"])[
        "predicted_units"
    ]
    .sum()
    .reset_index()
    .sort_values("predicted_units", ascending=False)
)

# 상위 5개 제품 추출
top_5_products_predictions = product_summary.head(5)

# 결과 표시
print(f"\nTotal Predicted Sales by Product for the {len(future_dates)}-day period:")
product_summary[["region", "item_type", "predicted_units"]] 

판매량이 가장 많을 것으로 예측된 상위 5개 제품은 다음과 같습니다

In [ ]:
for item in top_5_products_predictions["item_type"]:
    print(f"• {item}")

## 📊 예측 매출 시각화

이 그래프는 시간에 따른 각 제품의 주간 평균 예측 매출을 보여줍니다.

In [ ]:
lab_utilities.plot_forescast(top_5_products_predictions, product_sales, region)

***

## 🤖💡 생성형 AI 마케팅 캠페인 생성기 준비

이제 수익을 창출할 상위 5개 품목을 파악했으니, 마케팅 캠페인을 강화하기 위해 긍정적인 리뷰들을 추출해 보겠습니다.
리뷰 테이블을 쿼리합니다(아래에서 검색한 `project_ml_...` 값으로 `[REPLACE-VALUE-HERE]`를 변경합니다):

In [ ]:
print(project_db)

In [ ]:
%%sql project.athena
select * from "awsdatacatalog"."[REPLACE-VALUE-HERE]"."product_reviews"

판다스 DataFrame으로 쿼리 결과 변환하기

In [ ]:
reviews = _.to_pandas()

다음 워크샵 섹션에서 마케팅 캠페인을 생성하는 데 사용할 기본 생성형 AI 프롬프트입니다:


In [ ]:
base_text = """
다음 제품 데이터 활용:
제품명: {product_name}
설명: {product_description} 
리뷰: {product_reviews}
타겟 페르소나: 직장인

[타겟 페르소나]를 위한 맞춤형 마케팅 콘텐츠 제작:

1. 개인적 연결: 고객의 페인포인트/욕구를 활용한 감성적 연결
2. 제작:

* [타겟 페르소나]의 니즈를 다루는 맞춤 헤드라인
* 제품이 라이프스타일에 부합함을 강조하는 맞춤형 소셜 포스트 2개
* 고객 프로필에 맞는 맞춤형 혜택
* 이 고객 유형을 위한 구체적인 사용 사례
* 비슷한 고객들의 개인적인 후기
* 귀여운 이모티콘 사용 

톤: [타겟 페르소나]의 선호도에 맞춤
중점: 제품 혜택을 개인적 니즈와 연결
포함: 타겟팅된 솔루션과 공감할 수 있는 경험
"""

여기서는 각 제품 캠페인별로 하나의 프롬프트를 생성합니다. 다음 실습에서 사용해야 하므로 생성된 결과를 메모장에 복사해 두세요.

In [ ]:
# 프롬프트 생성하기
prompts = lab_utilities.generate_marketing_prompts(base_text, top_5_products_predictions, reviews)

# 프롬프트 출력하기
lab_utilities.print_marketing_prompts(prompts)